In [2]:
from docplex.mp.model import Model
import time
import numpy as np
def create_bpp(weights,bin_size):
    model = Model('BinPacking')
    n = len(weights)  # number of items
    m = n  # number of bins(worst case each item occupies one  bin)
    #Decision variable
    # x[i,j] = 1 if i occupies bin j
    x= model.binary_var_matrix(n,m,name='x', key_format='item_{0}_bin_{1}')
    # y[j]= 1 is bin j is used
    y=model.binary_var_list(m,name='y', key_format='bin_{0}') 
    # Objective: Minimize number of bins used
    model.minimize(model.sum(y[j] for j in range(m)))
    #Constraints:
    for i in range(n):
        item_assign=0
        for j in range(m):
            item_assign+=x[i,j]
    model.add_constraint(item_assign==1,f'item {i} assigned')

    for j in range(m):
        bin_capacity=0
        for i in range(n):
           bin_capacity+=weights[i]* x[i,j]
        model.add_constraint(bin_capacity<=bin_size*y[j],f'bin {j} capacity')
    
    return model, x, y   

    

In [3]:
import numpy as np

def ilp_to_qubo(model, x, y, weights, bin_size, A, B, C):
  
    n = len(weights)  # number of items
    m = n  # maximum number of bins
    Q = np.zeros((n*m + m, n*m + m))
    
    # Create variable to index mapping
    var_to_index = {}
    for i in range(n):
        for j in range(m):
            var_to_index[f'item_{i}_bin_{j}'] = i * m + j
    for j in range(m):
        var_to_index[f'bin_{j}'] = n * m + j
    
    # Process constraints from the model
    for constraint in model.iter_constraints():
        # Get left and right expressions
        left_expr = constraint.get_left_expr()
        right_expr = constraint.get_right_expr()
        sense=constraint.sense
        # Get the terms from the left expression
        terms = []
        for term in left_expr.iter_terms():
            var, coef = term[0], term[1]  # Unpack term tuple correctly
            var_name = var.get_name()
            if var_name in var_to_index:
                idx = var_to_index[var_name]
                terms.append((idx, coef))
        
       # Add quadratic penalties based on constraint type
        if sense == '==':  # equality constraint (one bin per item)
            # (sum x_ij - 1)^2 penalty
            for idx1, coef1 in terms:
                Q[idx1, idx1] += B
                for idx2, coef2 in terms:
                    if idx1 < idx2:
                        Q[idx1, idx2] += 2 * B
                        Q[idx2, idx1] += 2 * B
            
            # Add constant term to complete (sum x_ij - 1)^2
            for idx1, _ in terms:
                Q[idx1, idx1] -= 2 * B
        
        elif sense == '<=':  # inequality constraint (bin capacity)
            rhs = float(right_expr.constant if hasattr(right_expr, 'constant') else right_expr)
            
            # Add capacity constraint penalties
            for idx1, coef1 in terms:
                for idx2, coef2 in terms:
                    Q[idx1, idx2] += C * coef1 * coef2
                    
            # Add interaction with bin usage variable
            bin_idx = next((idx for idx, coef in terms if 'bin_' in var_to_index.keys()[idx]), None)
            if bin_idx is not None:
                for idx, coef in terms:
                    if idx != bin_idx:
                        Q[idx, bin_idx] -= C * coef * rhs
                        Q[bin_idx, idx] -= C * coef * rhs
                
                # Add squared term for bin capacity
                Q[bin_idx, bin_idx] += C * rhs * rhs
    
    # Add objective function (minimize number of bins)
    for j in range(m):
        Q[n*m + j, n*m + j] += A
        
    # Add additional penalties to ensure items are assigned
    for i in range(n):
        penalty = 1000  # Large penalty for not assigning items
        row_sum = 0
        for j in range(m):
            idx = var_to_index[f'item_{i}_bin_{j}']
            row_sum += 1
            Q[idx, idx] += penalty
        
        # Subtract penalty for correct assignment
        for j in range(m):
            idx = var_to_index[f'item_{i}_bin_{j}']
            Q[idx, idx] -= 2 * penalty * row_sum
            for k in range(j+1, m):
                idx2 = var_to_index[f'item_{i}_bin_{k}']
                Q[idx, idx2] += 2 * penalty
                Q[idx2, idx] += 2 * penalty
    
    return Q

In [5]:
def qubo_to_ising(Q):
    #converting QUBO matrix to ising hamiltonian
    n= Q.shape[0]
    h=np.zeros(n) #linear terms
    J=np.zeros((n,n)) #quadratic terms(couplings)
    constant= 0 #Constant term 
    for i in range(n):  
        for j in range(i,n):
            if i==j:
                h[i]+=Q[i,j]/2 #linear terms(left diagonal terms of qubo)
                constant += Q[i, i] / 4  # Extra factor for constant shift
            else:
                J[i,j]=Q[i,j]/4#symmetric distribution of off diagonal elements of qubo
                J[j,i]=Q[i,j]/4
                constant += Q[i, j] / 4  # Constant shift
    return h,J,constant

In [6]:
from qiskit.quantum_info import Pauli,SparsePauliOp
def hamiltonian(h,J,constant): 
    n= len(h)#number of qubits
    pauli_string=[]
    coeff=[]
    pauli_string.append(''.join(['I']*n))
    coeff.append(constant)
    for i in range(n): #adding single term qubits
       if abs(h[i])>0:
           pauli_str=['I']*n
           pauli_str[i]='Z'
           pauli_string.append(''.join(pauli_str))
           coeff.append(h[i])
    #add two-qubit terms(couplings)
    for i in range(n):
        for j in range (i+1,n):
            if abs(J[i,j])>0:
                pauli_str=['I']*n
                pauli_str[i]='Z'
                pauli_str[j]='Z'
                pauli_string.append(''.join(pauli_str))
                coeff.append(J[i,j])
    hamilton=0
    hamilton = SparsePauliOp(pauli_string, coeff)
    
    return hamilton

In [ ]:
#using variational approach
from qiskit import QuantumCircuit
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit import ParameterVector


def create_ansatz(nq,depth=3):
    ckt=QuantumCircuit(nq)
    param= ParameterVector('0',nq*depth*2)
    param_idx=0 # to track of the current position in params.
    for layer in range(depth):
         #single qubit rotation layer
         for i in range (nq):
             ckt.ry(param[param_idx],i)
             param_idx+=1
        #entangling layer with cz gate
         if layer < depth - 1:
          for i in range (nq-1):
            ckt.cz(i,i+1)
    return ckt



In [ ]:
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator as QasmEstimator
def run_vqe(h,J,constant,ansatz):
    nq = len(h)  # number of qubits is the length of h vector
    hamil=hamiltonian(h,J,constant)
    print(f"Hamiltonian: {hamil}")
    print(f"Number of Qubits: {nq}")
    print(f"Ansatz Parameters: {ansatz.num_parameters}")

    optimizer=COBYLA(maxiter=100)
    estimator = QasmEstimator()
        
    vqe = VQE(
            estimator=estimator,
            ansatz=ansatz,
            optimizer=optimizer,
            initial_point=[0.0] * ansatz.num_parameters  # Provide initial parameters
        )
    start_time = time.time()
    result = vqe.compute_minimum_eigenvalue(hamil)
    execution= time.time() - start_time
    return result.optimal_value,result.optimal_point,execution

In [ ]:
def solve(weights_list, bin_size):
    model, x, y = create_bpp(weights_list, bin_size)
    Q = ilp_to_qubo(model, x, y, weights_list, bin_size, A=1.0, B=100.0, C=100.0)
    h, J, constant = qubo_to_ising(Q)
    nq = len(h)
    ansatz=create_ansatz(nq,4)
    optimal_value, optimal_params, exec_time = run_vqe(h,J,constant, ansatz)

    
    return {
        'problem_size': len(weights_list),
        'optimal_value': optimal_value,
        'execution': exec_time,
        'parameters': optimal_params
    }

In [56]:
def instance():
    small=np.random.randint(1, 10, size=2)
    medium=np.random.randint(1,20,size=4)
    large=np.random.randint(1,50,size=7)
    bin_size=15
    results=[]
    
    result_1=solve(small,bin_size)
    print(f"Small instance results: {result_1}")
    result_2=solve(medium,bin_size)
    print(f"Medium instance results: {result_2}") 
   # result_3=solve(large,bin_size)
   # print(f"Large instance results: {result_3}")
    results.append({
            
            'small': result_1,
            'medium': result_2,
           #'large': large_result_3
        })
instance()
    
    
    
    
    

Hamiltonian: SparsePauliOp(['IIIIII', 'ZIIIII', 'IZIIII', 'IIZIII', 'IIIZII', 'IIIIZI', 'IIIIIZ', 'ZZIIII', 'IIZZII'],
              coeffs=[-1.9995e+03+0.j, -1.5000e+03+0.j, -1.5000e+03+0.j, -1.5000e+03+0.j,
 -1.5000e+03+0.j,  5.0000e-01+0.j,  5.0000e-01+0.j,  5.0000e+02+0.j,
  5.0000e+02+0.j])
Number of Qubits: 6
Ansatz Parameters: 24


/tmp/ipykernel_19869/2988429248.py:23: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = QasmEstimator()


Small instance results: {'problem_size': 2, 'optimal_value': -7000.1545950825675, 'execution': 0.4836921691894531, 'parameters': array([ 1.00000000e+00,  1.00000000e+00,  3.56151880e-17,  4.36632701e-17,
        2.11487149e-17,  7.18111864e-16,  1.00000000e+00,  1.00000000e+00,
        2.90447546e-17,  6.79320303e-19, -1.37513233e-16, -1.24478246e-16,
       -8.27540225e-17, -1.03579376e-16, -1.53313783e-16, -1.36027538e-16,
       -6.66030359e-17, -1.25728466e-16,  1.00000000e+00,  1.12499998e+00,
        7.44522965e-05,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])}
Hamiltonian: SparsePauliOp(['IIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIIIIII', 'IZIIIIIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIIIII', 'IIIZIIIIIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIIIII', 'IIIIIZIIIIIIIIIIIIII', 'IIIIIIZIIIIIIIIIIIII', 'IIIIIIIZIIIIIIIIIIII', 'IIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIZIIIII', 'IIIIII